<a href="https://colab.research.google.com/github/Guts-war/Guts-war/blob/main/Chatbot_para_Cria%C3%A7%C3%A3o_de_V%C3%ADdeos_no_YouTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q google-adk

import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
model = "gemini-2.0-flash"

# Função auxiliar para enviar mensagem ao agente e obter resposta
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

# Função auxiliar para exibir texto formatado
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Agente 1: Buscador de Conteúdo
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model=model,
        description="Busca conteúdos relevantes na web.",
        tools=[google_search],
        instruction="""
        Você é um agente de pesquisa na internet. Sua tarefa é usar a ferramenta de busca do Google
        (google_search) para encontrar os lançamentos e notícias mais recentes e relevantes sobre
        o tópico fornecido. Foque em conteúdos que sejam atuais, no máximo um mês antes da data de hoje.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    conteudos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return conteudos_buscados

# Agente 2: Selecionador de Conteúdo
def agente_selecionador(topico, conteudos_buscados):
    selecionador = Agent(
        name="agente_selecionador",
        model=model,
        description="Seleciona os conteúdos mais relevantes para o vídeo.",
        instruction="""
        Você é um especialista em conteúdo para YouTube. A partir dos conteúdos encontrados pelo
        agente de busca, você deve selecionar os mais relevantes e interessantes para um vídeo no YouTube
        sobre o tópico fornecido. Considere a relevância, o potencial de engajamento e a adequação ao público-alvo.
        Apresente os conteúdos selecionados em formato de lista, destacando os principais pontos de cada um.
        """,
    )
    entrada_do_agente_selecionador = f"Tópico: {topico}\nConteúdos buscados: {conteudos_buscados}"
    conteudos_selecionados = call_agent(selecionador, entrada_do_agente_selecionador)
    return conteudos_selecionados

# Agente 3: Criador de Roteiro
def agente_criador_roteiro(topico, conteudos_selecionados):
    criador_roteiro = Agent(
        name="agente_criador_roteiro",
        model=model,
        description="Cria um roteiro de vídeo para o YouTube.",
        instruction="""
        Você é um roteirista de vídeos para o YouTube. Com base nos conteúdos selecionados,
        crie um roteiro completo e detalhado para um vídeo no YouTube sobre o tópico fornecido.
        O roteiro deve incluir:
        - Título do vídeo
        - Introdução (gancho, apresentação do tema)
        - Desenvolvimento (divisão em seções, detalhes dos conteúdos, exemplos)
        - Ideias de elementos visuais (imagens, vídeos, gráficos) para cada seção
        - Sugestões de música de fundo e efeitos sonoros
        - Chamada para ação (CTA)
        - Encerramento
        Use uma linguagem clara, concisa e envolvente.
        """,
    )
    entrada_do_agente_criador_roteiro = f"Tópico: {topico}\nConteúdos selecionados: {conteudos_selecionados}"
    roteiro_do_video = call_agent(criador_roteiro, entrada_do_agente_criador_roteiro)
    return roteiro_do_video

# Agente 4: Produtor Multimídia
def agente_produtor_multimidia(topico, roteiro_do_video):
    produtor_multimidia = Agent(
        name="agente_produtor_multimidia",
        model=model,
        description="Gera os arquivos de multimídia para o vídeo.",
        instruction="""
        Você é um produtor de multimídia. A partir do roteiro do vídeo, você deve gerar os arquivos
        necessários para a produção do vídeo, incluindo:
        - Imagens (descreva as imagens necessárias para cada seção do vídeo)
        - Vídeos (descreva os vídeos necessários, se houver)
        - Áudios (descreva a música de fundo e os efeitos sonoros necessários)
        Especifique os tipos de arquivos, formatos e durações aproximadas.
        E informe links, de onde o conteúdo pode ser baixado.
        """,
    )
    entrada_do_agente_produtor_multimidia = f"Tópico: {topico}\nRoteiro do vídeo: {roteiro_do_video}"
    arquivos_multimidia = call_agent(produtor_multimidia, entrada_do_agente_produtor_multimidia)
    return arquivos_multimidia

# Função principal para executar o chatbot
def criar_video_youtube():
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    print("🚀 Iniciando o Sistema de Criação de Vídeos para o YouTube com 4 Agentes 🚀")
    topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o vídeo: ")

    if not topico:
        print("❌ Tópico inválido. Por favor, insira um tópico válido.")
        return

    print(f"Maravilha, vamos criar um vídeo sobre {topico}!")

    conteudos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Resultado do Agente 1 (Buscador de Conteúdo) ---\n")
    display(to_markdown(conteudos_buscados))
    print("---------------------------------\n")

    conteudos_selecionados = agente_selecionador(topico, conteudos_buscados)
    print("\n--- Resultado do Agente 2 (Selecionador de Conteúdo) ---\n")
    display(to_markdown(conteudos_selecionados))
    print("---------------------------------\n")

    roteiro_do_video = agente_criador_roteiro(topico, conteudos_selecionados)
    print("\n--- Resultado do Agente 3 (Criador de Roteiro) ---\n")
    display(to_markdown(roteiro_do_video))
    print("---------------------------------\n")

    arquivos_multimidia = agente_produtor_multimidia(topico, roteiro_do_video)
    print("\n--- Resultado do Agente 4 (Produtor Multimídia) ---\n")
    display(to_markdown(arquivos_multimidia))
    print("---------------------------------\n")

if __name__ == "__main__":
    criar_video_youtube()

🚀 Iniciando o Sistema de Criação de Vídeos para o YouTube com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o vídeo: Erros do Concílio de Trento
Maravilha, vamos criar um vídeo sobre Erros do Concílio de Trento!

--- Resultado do Agente 1 (Buscador de Conteúdo) ---



> Para responder à sua pergunta sobre "Erros do Concílio de Trento", preciso buscar informações atualizadas sobre este tema, focando em análises ou discussões recentes que possam ter surgido. Considerando que o Concílio de Trento é um evento histórico, a busca será direcionada a interpretações ou críticas modernas.
> 
> 
> Com base nas informações encontradas, o Concílio de Trento, realizado entre 1545 e 1563, foi um marco na história da Igreja Católica, principalmente em resposta à Reforma Protestante. Em vez de focar em "erros", é mais preciso analisar as críticas e reinterpretações modernas sobre o concílio.
> 
> *   **Reafirmação de Dogmas e Doutrinas:** O Concílio reafirmou dogmas católicos como os sete sacramentos, a autoridade papal, a salvação pelas obras, o culto aos santos e a doutrina da transubstanciação (Toda Matéria, Brasil Escola, Concílio Ecumênico de Trento - MONTFORT).
> *   **Controle e Disciplina:** O Concílio de Trento estabeleceu um sistema de controle social através da necessidade de confissão sistemática dos pecados (Uma Igreja, dois mundos: análise das diretrizes conciliares apresentadas em Trento e em Lima (séc).
> *   **Formação do Clero:** Foi dada grande ênfase à formação do clero, com a criação de seminários (O CONCÍLIO DE TRENTO E A FORMAÇÃO SACERDOTAL TRIDENTINA NA AMÉRICA PORTUGUESA).
> *   **Relação com a Modernidade:** O Concílio precisou enfrentar valores da modernidade, como a negação de dogmas e a prevalência da razão (O CONCÍLIO DE TRENTO E A FORMAÇÃO SACERDOTAL TRIDENTINA NA AMÉRICA PORTUGUESA).
> *   **Críticas Modernas:** Alguns autores questionam o caráter dogmático de certas decisões do Concílio, como a indissolubilidade do matrimônio (O Concílio de Trento e a indissolubilidade do matrimónio).
> *   **Modelo Eclesial:** Há debates sobre o modelo de Igreja que emergiu do Concílio de Trento, visto como tradicional, centralista e de confronto, em contraste com o modelo do Concílio Vaticano II, que busca maior diálogo e abertura (Os trabalhos do novo Papa e os debates para o futuro do catolicismo - Sete Margens).
> *   **Relevância Contínua:** Apesar das críticas, alguns argumentam que os preceitos do Concílio de Trento continuam relevantes, enquanto outros defendem uma revisão dos modelos de organização eclesial (II Concílio do Vaticano: A luz de Trento ainda ilumina a Igreja contemporânea, A longa sombra de Trento: a crise dos abusos, dos seminários, das dioceses e do laicato. Artigo de Massimo Faggioli - IHU).
> 
> É importante notar que a interpretação do Concílio de Trento é um tema complexo e multifacetado, com diversas perspectivas e análises ao longo da história.
> 


---------------------------------


--- Resultado do Agente 2 (Selecionador de Conteúdo) ---



> ## Sugestões de Conteúdo para Vídeo no YouTube: "Erros" do Concílio de Trento (Uma Análise Crítica)
> 
> Considerando o material encontrado, sugiro os seguintes pontos para estruturar um vídeo interessante e informativo sobre o Concílio de Trento, evitando focar em "erros" (já que o termo pode ser carregado) e optando por uma análise crítica:
> 
> 1.  **Introdução:**
>     *   Contextualize o Concílio de Trento: Reforma Protestante, crise na Igreja Católica.
>     *   Explique que, em vez de "erros", o vídeo abordará as **críticas e reinterpretações modernas** do Concílio.
>     *   Apresente a importância do Concílio para a Igreja Católica e seu impacto duradouro.
> 
> 2.  **Reafirmações Dogmáticas e Doutrinárias:**
>     *   Liste e explique os principais dogmas e doutrinas reafirmadas:
>         *   Sete Sacramentos
>         *   Autoridade Papal
>         *   Salvação pelas Obras
>         *   Culto aos Santos
>         *   Transubstanciação
>     *   Questione: Por que essas reafirmações foram necessárias? Qual o impacto delas na época e hoje?
> 
> 3.  **Controle Social e Disciplina:**
>     *   Explore o sistema de controle social implementado através da confissão sistemática.
>     *   Debate: Esse controle era necessário? Quais as consequências positivas e negativas?
> 
> 4.  **Formação do Clero:**
>     *   Destaque a importância da criação de seminários e a ênfase na formação do clero.
>     *   Questione: Essa ênfase contribuiu para a qualidade do clero? Como a formação do clero evoluiu desde então?
> 
> 5.  **O Concílio e a Modernidade:**
>     *   Analise como o Concílio lidou com os valores emergentes da modernidade (razão, ciência, etc.).
>     *   Debate: O Concílio se adaptou bem à modernidade? Que tensões surgiram?
> 
> 6.  **Críticas Modernas e Reinterpretações:**
>     *   Apresente as principais críticas modernas ao Concílio:
>         *   Caráter dogmático de algumas decisões (ex: indissolubilidade do matrimônio)
>         *   Modelo de Igreja centralista e de confronto
>     *   Inclua diferentes perspectivas de autores e estudiosos modernos.
> 
> 7.  **O Legado do Concílio de Trento:**
>     *   Discuta a relevância contínua dos preceitos do Concílio.
>     *   Questione: Os modelos de organização eclesial precisam ser revisados à luz do Concílio Vaticano II?
> 
> 8.  **Conclusão:**
>     *   Reafirme a complexidade e multifacetamento do tema.
>     *   Incentive os espectadores a pesquisarem e formarem suas próprias opiniões.
>     *   Lance perguntas para debate nos comentários: Qual sua opinião sobre o Concílio de Trento? Qual o impacto dele na Igreja Católica hoje?
> 
> **Observações Importantes:**
> 
> *   **Tom:** Mantenha um tom informativo e objetivo, evitando juízos de valor. O objetivo é apresentar diferentes perspectivas e estimular o debate.
> *   **Recursos Visuais:** Utilize imagens de época, mapas, citações de documentos relevantes e entrevistas com especialistas para enriquecer o vídeo.
> *   **Público-Alvo:** Considere o nível de conhecimento do seu público sobre história da Igreja e teologia. Adapte a linguagem e a profundidade da análise de acordo.
> *   **Fontes:** Mencione todas as fontes utilizadas na descrição do vídeo para dar credibilidade ao conteúdo.
> *   **SEO:** Utilize palavras-chave relevantes no título, descrição e tags do vídeo para otimizar a busca no YouTube.
> 
> Ao seguir essas sugestões, você poderá criar um vídeo informativo, interessante e que promova uma reflexão crítica sobre um dos eventos mais importantes da história da Igreja Católica.
> 


---------------------------------


--- Resultado do Agente 3 (Criador de Roteiro) ---



> ## Roteiro de Vídeo para YouTube: Concílio de Trento: Críticas e Reinterpretações Modernas
> 
> **Título:** Concílio de Trento: Reafirmações, Críticas e o Legado na Igreja Católica de Hoje
> 
> **Miniatura:**
> *   Imagem: Uma representação artística do Concílio de Trento com um filtro moderno e reflexivo.
> *   Texto: Concílio de Trento: O que ficou? O que mudou?
> 
> **Música de Fundo:**
> *   Introdução: Música clássica suave e reflexiva.
> *   Desenvolvimento: Variações na intensidade da música de acordo com o tema (mais tensa em críticas, mais suave em legados).
> *   Conclusão: Música inspiradora e convidativa ao debate.
> 
> **Introdução (0:00 - 1:00)**
> 
> *   **Visual:**
>     *   Abertura com imagens da Europa no século XVI: Reforma Protestante, mapas da época, retratos de Lutero e outros reformadores.
>     *   Logotipo do canal e título do vídeo.
> *   **Áudio:**
>     *   **[Música suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Olá, pessoal! Sejam bem-vindos ao canal. No vídeo de hoje, vamos mergulhar em um dos eventos mais importantes e controversos da história da Igreja Católica: o Concílio de Trento."
>         *   "Em meados do século XVI, a Igreja enfrentava uma crise sem precedentes com a Reforma Protestante. Em resposta, o Papa Paulo III convocou o Concílio de Trento. Longe de ser um mero evento histórico, o Concílio moldou a Igreja Católica pelos séculos seguintes."
>         *   "Hoje, ao invés de focar em 'erros', vamos analisar criticamente as principais decisões do Concílio, as críticas que surgiram ao longo do tempo e como suas decisões são reinterpretadas no mundo contemporâneo."
>         *   "Preparem-se para uma jornada de reflexão e debate. Se você se interessa por história da Igreja e teologia, este vídeo é para você! Já deixe seu like e inscreva-se no canal para não perder os próximos vídeos."
> 
> **Reafirmações Dogmáticas e Doutrinárias (1:00 - 4:00)**
> 
> *   **Visual:**
>     *   Imagens dos Sete Sacramentos (batismo, eucaristia, etc.).
>     *   Representação da Autoridade Papal (brasões, fotos do Vaticano).
>     *   Ilustrações do culto aos santos e da Transubstanciação.
>     *   Gráfico comparando as visões católicas e protestantes sobre a salvação.
> *   **Áudio:**
>     *   **[Música de fundo com ritmo moderado]**
>     *   **[Efeito sonoro suave ao trocar de tópico]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "O Concílio de Trento foi um momento crucial de reafirmação da doutrina católica. Em resposta aos protestantes, o Concílio reafirmou os Sete Sacramentos, a autoridade do Papa, a importância das obras para a salvação, o culto aos santos e a doutrina da Transubstanciação."
>         *   "Vamos analisar cada um desses pontos. Por que a Igreja sentiu a necessidade de reafirmá-los? Qual era a visão dos protestantes? E, mais importante, qual o impacto dessas reafirmações na vida dos fiéis e na estrutura da Igreja?"
>         *   "Por exemplo, a reafirmação da Transubstanciação, a crença de que o pão e o vinho se transformam no corpo e sangue de Cristo, foi uma resposta direta à visão simbólica da Ceia defendida por alguns reformadores. Essa decisão moldou a liturgia e a teologia eucarística católica por séculos."
>         *   "**[Inserir fala de um especialista (vídeo curto ou áudio)]**: 'A reafirmação da autoridade papal no Concílio de Trento solidificou o poder centralizado da Igreja, mas também gerou tensões e críticas sobre o autoritarismo'."
> 
> **Controle Social e Disciplina (4:00 - 6:00)**
> 
> *   **Visual:**
>     *   Imagens de confessionários e cenas de confissão (pinturas, ilustrações).
>     *   Representações de regras e regulamentos da época.
>     *   Infográficos sobre o sistema de confissão sistemática.
> *   **Áudio:**
>     *   **[Música de fundo com tom um pouco mais tenso]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Além das questões doutrinárias, o Concílio também implementou um sistema de controle social e disciplina através da confissão sistemática. A confissão se tornou uma ferramenta para monitorar a conduta dos fiéis e garantir a adesão aos preceitos da Igreja."
>         *   "Esse controle era necessário? Quais as consequências positivas e negativas? A confissão ajudou a fortalecer a fé e a moralidade, ou gerou um clima de medo e opressão?"
>         *   "É importante ressaltar que o sistema de confissão também oferecia um espaço para o perdão e a reconciliação, mas é inegável que ele também representava uma forma de vigilância e controle social."
> 
> **Formação do Clero (6:00 - 8:00)**
> 
> *   **Visual:**
>     *   Imagens de seminários antigos e modernos.
>     *   Representações de aulas e estudos teológicos.
>     *   Fotos de padres e líderes religiosos em diferentes épocas.
> *   **Áudio:**
>     *   **[Música de fundo com tom mais positivo]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Uma das decisões mais importantes do Concílio foi a criação de seminários para a formação do clero. Antes, a formação dos padres era muitas vezes inadequada, o que contribuía para a crise da Igreja. Com os seminários, a Igreja buscou garantir um clero mais instruído e preparado para lidar com os desafios da época."
>         *   "Essa ênfase na formação do clero contribuiu para a qualidade do clero? Como a formação do clero evoluiu desde então? Os seminários ainda são relevantes hoje em dia?"
>         *   "**[Inserir depoimento de um padre ou seminarista (vídeo curto ou áudio)]**: 'A formação em um seminário é fundamental para preparar os futuros padres para o serviço à Igreja e à comunidade.'"
> 
> **O Concílio e a Modernidade (8:00 - 10:00)**
> 
> *   **Visual:**
>     *   Imagens de cientistas e filósofos da época moderna (Galileu, Descartes, etc.).
>     *   Representações de avanços científicos e tecnológicos.
>     *   Gráfico comparando as visões do Concílio e da modernidade sobre a razão e a ciência.
> *   **Áudio:**
>     *   **[Música de fundo com tom um pouco mais complexo]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Como o Concílio lidou com os valores emergentes da modernidade, como a razão e a ciência? O Concílio se adaptou bem à modernidade? Que tensões surgiram?"
>         *   "É importante lembrar que o Concílio ocorreu em um período de grandes transformações intelectuais e científicas. A Igreja, por um lado, buscava preservar a sua doutrina e autoridade. Por outro, precisava lidar com as novas descobertas e ideias que desafiavam a sua visão do mundo."
>         *   "Essa tensão entre fé e razão, entre tradição e modernidade, ainda é relevante hoje em dia."
> 
> **Críticas Modernas e Reinterpretações (10:00 - 13:00)**
> 
> *   **Visual:**
>     *   Citações de autores e estudiosos modernos que criticam o Concílio.
>     *   Imagens de debates e discussões teológicas.
>     *   Representações de diferentes modelos de Igreja.
> *   **Áudio:**
>     *   **[Música de fundo com tom crítico e reflexivo]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "O Concílio de Trento não escapou das críticas modernas. Alguns autores questionam o caráter dogmático de algumas decisões, como a indissolubilidade do matrimônio. Outros criticam o modelo de Igreja centralista e de confronto que emergiu do Concílio."
>         *   "É importante apresentar diferentes perspectivas e considerar os argumentos de cada lado. Quais são as principais críticas ao Concílio? Elas são justificadas? Como podemos reinterpretar as decisões do Concílio à luz do Concílio Vaticano II?"
>         *   "**[Inserir trechos de entrevistas com teólogos ou historiadores (vídeos curtos ou áudios)]**: 'O Concílio de Trento, embora importante para a Igreja, precisa ser contextualizado e reinterpretado à luz das mudanças sociais e culturais dos últimos séculos.'"
> 
> **O Legado do Concílio de Trento (13:00 - 15:00)**
> 
> *   **Visual:**
>     *   Imagens da Igreja Católica atual.
>     *   Representações de rituais e práticas religiosas modernas.
>     *   Gráfico mostrando a evolução da Igreja Católica desde o Concílio de Trento.
> *   **Áudio:**
>     *   **[Música de fundo com tom inspirador e reflexivo]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Qual o legado do Concílio de Trento para a Igreja Católica hoje? As decisões do Concílio ainda são relevantes? Os modelos de organização eclesial precisam ser revisados à luz do Concílio Vaticano II?"
>         *   "É inegável que o Concílio de Trento moldou a Igreja Católica por séculos. Muitas das suas decisões ainda são seguidas hoje em dia. No entanto, é importante reconhecer que a Igreja evoluiu e se adaptou às mudanças do mundo."
>         *   "O Concílio Vaticano II, por exemplo, promoveu uma maior abertura e diálogo com outras religiões e com o mundo moderno. É fundamental que a Igreja continue a refletir sobre o seu passado e a buscar novas formas de responder aos desafios do presente."
> 
> **Conclusão (15:00 - 16:00)**
> 
> *   **Visual:**
>     *   Retorno à imagem da miniatura.
>     *   Tela final com informações do canal e links para outros vídeos.
> *   **Áudio:**
>     *   **[Música inspiradora]**
>     *   **[Efeito sonoro suave]**
>     *   **Roteiro:**
>         *   **[Apresentador(a)]:** "Chegamos ao final do nosso vídeo sobre o Concílio de Trento. Como vimos, este foi um evento complexo e multifacetado que teve um impacto profundo na história da Igreja Católica."
>         *   "Espero que este vídeo tenha sido útil para vocês. Se gostaram, deixem o seu like, compartilhem com seus amigos e inscrevam-se no canal para não perder os próximos vídeos."
>         *   "E agora, queremos saber a sua opinião: Qual sua opinião sobre o Concílio de Trento? Qual o impacto dele na Igreja Católica hoje? Deixem seus comentários abaixo. Vamos debater!"
>         *   "Até a próxima!"
> 
> **Chamada para Ação (CTA):**
> 
> *   **Visual:**
>     *   Animação gráfica com os botões de "Like", "Inscrever-se" e "Compartilhar".
>     *   Cards com links para outros vídeos relacionados.
> *   **Áudio:**
>     *   **[Apresentador(a)]:** "Se você gostou deste vídeo, não se esqueça de deixar o seu like, inscrever-se no canal e compartilhar com seus amigos. E não deixe de conferir os outros vídeos do canal sobre história da Igreja e teologia."
> 
> **SEO:**
> 
> *   **Título:** Concílio de Trento: Reafirmações, Críticas e o Legado na Igreja Católica de Hoje
> *   **Descrição:** Uma análise crítica do Concílio de Trento, suas principais decisões, as críticas modernas e o legado na Igreja Católica atual. Descubra como este evento moldou a Igreja e o que ainda é relevante hoje.
> *   **Tags:** Concílio de Trento, Igreja Católica, Reforma Protestante, História da Igreja, Teologia, Dogmas, Doutrinas, Críticas, Reinterpretações, Concílio Vaticano II, Reafirmações, Autoridade Papal, Sacramentos, Formação do Clero, Modernidade, Controle Social.
> 
> **Observações Adicionais:**
> 
> *   Adaptar a linguagem ao público-alvo.
> *   Utilizar legendas para acessibilidade.
> *   Garantir boa qualidade de áudio e vídeo.
> *   Interagir com os comentários e responder às perguntas dos espectadores.
> 
> Este roteiro oferece uma estrutura completa para um vídeo informativo e envolvente sobre o Concílio de Trento. Ao seguir estas sugestões, você poderá criar um conteúdo de alta qualidade que atraia e engaje o seu público.
> 


---------------------------------


--- Resultado do Agente 4 (Produtor Multimídia) ---



> Ok, aqui estão os arquivos multimídia necessários para a produção do vídeo sobre o Concílio de Trento, baseados no roteiro fornecido:
> 
> **I. Visuais**
> 
> 1.  **Abertura:**
>     *   **Tipo:** Vídeo
>     *   **Formato:** MP4
>     *   **Duração:** 10 segundos
>     *   **Descrição:** Montagem de imagens da Europa no século XVI: Reforma Protestante, mapas da época, retratos de Lutero e outros reformadores. Movimentos sutis de câmera e transições suaves.
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/) (imagens e vídeos de arquivo)
>         *   [https://pixabay.com/](https://pixabay.com/) (imagens e vídeos gratuitos)
>     *   **Música:** Abertura de fundo (item III.1)
> 
> 2.  **Logotipo e Título:**
>     *   **Tipo:** Vídeo/Animação
>     *   **Formato:** MP4 (com transparência, se possível)
>     *   **Duração:** 5 segundos
>     *   **Descrição:** Animação simples do logotipo do canal, seguido pelo título do vídeo sobreposto.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 3.  **Imagens dos Sete Sacramentos:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 7 imagens
>     *   **Descrição:** Representações artísticas clássicas dos sete sacramentos (Batismo, Confirmação, Eucaristia, Penitência, Unção dos Enfermos, Ordem e Matrimônio).
>     *   **Onde encontrar:**
>         *   [https://www.wga.hu/](https://www.wga.hu/) (Web Gallery of Art - domínio público)
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/) (Wikimedia Commons - verificar licenças)
> 
> 4.  **Autoridade Papal:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 2-3 imagens
>     *   **Descrição:** Brasões papais, fotos do Vaticano (fachada, interior da Basílica de São Pedro).
>     *   **Onde encontrar:**
>         *   [https://www.vatican.va/](https://www.vatican.va/) (site oficial do Vaticano - imagens podem ter restrições)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 5.  **Culto aos Santos e Transubstanciação:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 2-3 imagens
>     *   **Descrição:** Imagens de santos venerados, representações artísticas da Eucaristia e da Transubstanciação.
>     *   **Onde encontrar:**
>         *   [https://www.wga.hu/](https://www.wga.hu/)
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/)
> 
> 6.  **Gráfico Comparativo (Salvação):**
>     *   **Tipo:** Imagem
>     *   **Formato:** PNG/JPG
>     *   **Descrição:** Gráfico simples comparando as visões católica e protestante sobre a salvação (fé vs. fé + obras).
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 7.  **Confessionários e Confissão:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 2-3 imagens
>     *   **Descrição:** Pinturas ou ilustrações de confessionários antigos, cenas de confissão.
>     *   **Onde encontrar:**
>         *   [https://www.wga.hu/](https://www.wga.hu/)
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/)
> 
> 8.  **Regras e Regulamentos:**
>     *   **Tipo:** Imagem
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Representações de documentos antigos, pergaminhos, ou textos que simbolizem regras e regulamentos da época.
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 9.  **Infográfico (Confissão):**
>     *   **Tipo:** Imagem
>     *   **Formato:** PNG
>     *   **Descrição:** Infográfico sobre o sistema de confissão sistemática, explicando seu funcionamento e propósito.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 10. **Seminários (Antigos e Modernos):**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 2-4 imagens
>     *   **Descrição:** Imagens de seminários antigos (arquitetura, salas de aula) e modernos.
>     *   **Onde encontrar:**
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/)
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
> 
> 11. **Aulas e Estudos Teológicos:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Representações de aulas e estudos teológicos (estudantes lendo, debatendo).
>     *   **Onde encontrar:**
>         *   [https://www.wga.hu/](https://www.wga.hu/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 12. **Padres e Líderes Religiosos:**
>     *   **Tipo:** Fotos
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 3-5 fotos
>     *   **Descrição:** Fotos de padres e líderes religiosos em diferentes épocas.
>     *   **Onde encontrar:**
>         *   Arquivos da Igreja (se possível)
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/)
> 
> 13. **Cientistas e Filósofos (Modernidade):**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Quantidade:** 3-4 imagens
>     *   **Descrição:** Retratos de cientistas e filósofos da época moderna (Galileu, Descartes, etc.).
>     *   **Onde encontrar:**
>         *   [https://www.wga.hu/](https://www.wga.hu/)
>         *   [https://commons.wikimedia.org/](https://commons.wikimedia.org/)
> 
> 14. **Avanços Científicos e Tecnológicos:**
>     *   **Tipo:** Imagens/Vídeos curtos
>     *   **Formato:** JPG/PNG/MP4
>     *   **Descrição:** Representações de avanços científicos e tecnológicos da época (telescópios, mapas celestes, etc.).
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 15. **Gráfico Comparativo (Razão e Ciência):**
>     *   **Tipo:** Imagem
>     *   **Formato:** PNG
>     *   **Descrição:** Gráfico comparando as visões do Concílio e da modernidade sobre a razão e a ciência.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 16. **Citações de Autores Modernos:**
>     *   **Tipo:** Imagens
>     *   **Formato:** PNG
>     *   **Descrição:** Citações curtas de autores e estudiosos modernos que criticam o Concílio, com seus nomes e referências. Design limpo e legível.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 17. **Debates e Discussões Teológicas:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Imagens de pessoas debatendo, livros, ambientes de discussão.
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 18. **Modelos de Igreja:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Representações visuais de diferentes modelos de Igreja (tradicional, moderna, etc.). Podem ser mais abstratas e simbólicas.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 19. **Igreja Católica Atual:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Imagens da Igreja Católica atual (pessoas em missa, atividades sociais, etc.).
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 20. **Rituais e Práticas Religiosas Modernas:**
>     *   **Tipo:** Imagens
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Representações de rituais e práticas religiosas modernas (celebrações, obras de caridade, etc.).
>     *   **Onde encontrar:**
>         *   [https://www.shutterstock.com/](https://www.shutterstock.com/)
>         *   [https://pixabay.com/](https://pixabay.com/)
> 
> 21. **Gráfico (Evolução da Igreja):**
>     *   **Tipo:** Imagem
>     *   **Formato:** PNG
>     *   **Descrição:** Gráfico mostrando a evolução da Igreja Católica desde o Concílio de Trento até os dias atuais.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 22. **Miniatura (Final):**
>     *   **Tipo:** Imagem
>     *   **Formato:** JPG/PNG
>     *   **Descrição:** Imagem da miniatura usada no início do vídeo, com o título e o logo do canal.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 23. **Tela Final:**
>     *   **Tipo:** Vídeo
>     *   **Formato:** MP4
>     *   **Duração:** 5-10 segundos
>     *   **Descrição:** Tela final com informações do canal, links para outros vídeos e redes sociais.
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> 24. **Chamada para Ação (CTA):**
>     *   **Tipo:** Vídeo/Animação
>     *   **Formato:** MP4 (com transparência, se possível)
>     *   **Duração:** 5 segundos
>     *   **Descrição:** Animação gráfica com os botões de "Like", "Inscrever-se" e "Compartilhar".
>     *   **Onde encontrar:** Criado pelo designer gráfico do canal.
> 
> **II. Áudios**
> 
> 1.  **Música de Fundo (Introdução):**
>     *   **Tipo:** Música
>     *   **Formato:** MP3/WAV
>     *   **Duração:** 1 minuto
>     *   **Descrição:** Música clássica suave e reflexiva.
>     *   **Onde encontrar:**
>         *   [https://www.youtube.com/audiolibrary/](https://www.youtube.com/audiolibrary/) (Biblioteca de áudio do YouTube - gratuita)
>         *   [https://www.epidemicsound.com/](https://www.epidemicsound.com/) (paga, com licença para uso comercial)
> 
> 2.  **Música de Fundo (Desenvolvimento):**
>     *   **Tipo:** Música
>     *   **Formato:** MP3/WAV
>     *   **Duração:** Variável (acompanha o vídeo)
>     *   **Descrição:** Variações na intensidade da música de acordo com o tema (mais tensa em críticas, mais suave em legados). Pode ser a mesma música da introdução, com diferentes arranjos.
>     *   **Onde encontrar:** Mesmas fontes da música de introdução.
> 
> 3.  **Música de Fundo (Conclusão):**
>     *   **Tipo:** Música
>     *   **Formato:** MP3/WAV
>     *   **Duração:** 1 minuto
>     *   **Descrição:** Música inspiradora e convidativa ao debate.
>     *   **Onde encontrar:** Mesmas fontes da música de introdução.
> 
> 4.  **Efeitos Sonoros:**
>     *   **Tipo:** Efeitos Sonoros
>     *   **Formato:** MP3/WAV
>     *   **Descrição:** Efeitos sonoros suaves para transições entre tópicos, realçar momentos importantes, etc. (ex: sino, folhear de páginas).
>     *   **Onde encontrar:**
>         *   [https://www.zapsplat.com/](https://www.zapsplat.com/) (gratuito e pago)
>         *   [https://freesound.org/](https://freesound.org/) (gratuito)
> 
> 5.  **Fala do Especialista/Padre/Seminarista:**
>     *   **Tipo:** Áudio/Vídeo
>     *   **Formato:** MP3/MP4
>     *   **Duração:** 15-30 segundos cada
>     *   **Descrição:** Trechos de entrevistas ou depoimentos em áudio ou vídeo.
>     *   **Onde encontrar:** A ser gravado ou obtido com os especialistas.
> 
> **III. Outros**
> 
> *   **Roteiro Narrado:**
>     *   **Tipo:** Áudio
>     *   **Formato:** MP3/WAV
>     *   **Descrição:** Narração completa do roteiro pelo apresentador(a).
>     *   **Onde encontrar:** A ser gravado pelo apresentador(a).
> 
> **Observações:**
> 
> *   **Licenças:** Verifique sempre as licenças de uso das imagens, vídeos e músicas antes de utilizá-los.
> *   **Design Gráfico:** O designer gráfico do canal será responsável pela criação de gráficos, animações e outros elementos visuais personalizados.
> *   **Edição:** A edição de vídeo é crucial para juntar todos os elementos de forma coesa e criar um produto final de alta qualidade.
> *   **Onde Encontrar:** Os links fornecidos são apenas sugestões. Existem muitos outros sites e recursos disponíveis online.
> 
> Este plano detalhado deve fornecer uma base sólida para a produção do vídeo sobre o Concílio de Trento. Lembre-se de adaptar os recursos às necessidades e ao estilo do seu canal.


---------------------------------

